In [11]:
!py -m pip install langgraph-checkpoint-sqlite

# !py -m pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-openai langchain-chroma bs4

  Attempting uninstall: langgraph-checkpoint
    Found existing installation: langgraph-checkpoint 2.0.9
    Uninstalling langgraph-checkpoint-2.0.9:
      Successfully uninstalled langgraph-checkpoint-2.0.9



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings
import pandas as pd
import os
import shutil
import numpy as np
from uuid import uuid4
from langchain_community.document_loaders import PyPDFLoader, UnstructuredWordDocumentLoader
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain.prompts import ChatPromptTemplate
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.docstore.document import Document
from typing import List, Tuple
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langgraph.store.base import BaseStore
from langchain_core.messages import SystemMessage, HumanMessage
import re
from typing import Annotated, TypedDict
import operator
from langchain_core.documents import Document

In [2]:
OPENAI_DEPLOYMENT_ENDPOINT = "https://advancedanalyticsopenaikey.openai.azure.com/"
OPENAI_DEPLOYMENT_ENDPOINT_embed = "https://pkl-aa-dev-aiservices.openai.azure.com/" 
OPENAI_API_KEY = "FqFd4DBx1W97MSVjcZvdQsmQlhI80hXjl48iWYmZ4W3NutUlWvf0JQQJ99BDACYeBjFXJ3w3AAABACOGl3xo" 
OPENAI_API_VERSION = "2024-12-01-preview"
OPENAI_API_KEY_EMBEDDINGS = "AXEC3y1jC9ZNGCBB12NZwrpBSzScq1esexgvCXiqw7PaHE04vSMbJQQJ99BDACYeBjFXJ3w3AAABACOG4CMN" 
OPENAI_DEPLOYMENT_NAME = "gpt-4o"
OPENAI_MODEL_NAME="gpt-4o"
embedding_api_version = "2024-02-01"

# OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
# CHROMA_PATH = "c:\\Users\\SujaySunilNagvekar\\VM\\GEN AI\\SERFF\\vector_db\\testing_db"
DB_PATH  = "C:\\Users\\SujaySunilNagvekar\\VM\\GEN AI\\KM\\vector_db"
DATA_FOLDER = "C:\\Users\\SujaySunilNagvekar\\VM\\GEN AI\\KM\\Documents"

### Cohere API key 
# cohere_API_key = "wHyiTViP32Y3Q8Qwhjmd4QGNCkYNpxqtsemtSri3"
# co = cohere.Client(cohere_API_key)

In [3]:

llm = AzureChatOpenAI(
                        temperature=0.7,
                        deployment_name=OPENAI_DEPLOYMENT_NAME,
                        model_name=OPENAI_MODEL_NAME,
                        azure_endpoint=OPENAI_DEPLOYMENT_ENDPOINT,
                        openai_api_version=OPENAI_API_VERSION,
                        openai_api_key=OPENAI_API_KEY            
                    )

embeddings = AzureOpenAIEmbeddings(
                        deployment="text-embedding-3-small",
                        model="text-embedding-3-small",
                        azure_endpoint=OPENAI_DEPLOYMENT_ENDPOINT_embed,
                        openai_api_version=embedding_api_version,
                        openai_api_key=OPENAI_API_KEY_EMBEDDINGS)


In [17]:
import pandas as pd
from tqdm import tqdm
from langchain_core.messages import SystemMessage, HumanMessage
from pydantic import BaseModel, Field
from typing import Optional
import random
from time import time
import unicodedata

def normalize_text(text):
    return unicodedata.normalize("NFKC", text).encode("utf-8", errors="ignore").decode("utf-8")

def fix_smart_quotes(text):
    return (
        text.replace("‘", "'").replace("’", "'")
            .replace("“", '"').replace("”", '"')
            .replace("–", "-").replace("—", "-")
    )


# Load your CSV
df = pd.read_csv("C://Users//SujaySunilNagvekar//VM//GEN AI//KM//new_claims_data.csv", encoding="cp1252")
# Load policy data
policy_df = pd.read_csv("C://Users//SujaySunilNagvekar//VM//GEN AI//KM//new_policy_data.csv", encoding="cp1252")


class ClaimNotes(BaseModel):
    fnol_call: str = Field(description="Transcript of the FNOL call between customer and agent.")
    adjuster_notes: str = Field(description="Adjuster's notes in bullet or paragraph format.")

# ------------------------------
# Prompt Template
# ------------------------------
# Prompts options for variation
tones = [
    "Be warm and empathetic, make the agent sound reassuring.",
    "Be brief and to the point, like a rushed weekday call.",
    "Include clarifying questions by the agent.",
    "Make the customer slightly emotional but cooperative.",
    "Add a follow-up suggestion by the agent.",
    "Agent is confident and efficient, but polite.",
    "Agent uses layman’s language to explain coverage.",
    "Agent deals with a mildly frustrated customer.",
    "Agent reassures about payment processing.",
    "Include a moment where the customer expresses gratitude."
]

note_styles = [
    "Use bullet points for adjuster notes.",
    "Use a narrative paragraph style.",
    "Include a timestamp and adjuster name.",
    "Add follow-up actions or pending steps in the notes.",
    "Summarize with a conclusion at the end."
]

note_scenarios = [
    "This is an initial FNOL entry. Just summarize what was reported and next steps.",
    "The claim is under review. Include pending documentation, follow-ups, and internal status.",
    "This is a follow-up after initial processing. Include updated estimate or new details.",
    "The claim has been resolved. Summarize the decision and any payment issued.",
    "There is a delay or irregularity in the claim. Highlight concerns or next steps.",
    "Claim was reopened due to new evidence or customer request.",
    "Customer was contacted multiple times, summarize call attempts or missed connections.",
    "Adjuster flagged the need for internal legal or fraud team to assess the situation.",
    "Include interaction with a 3rd party (e.g., Safelite, towing company, repair contractor)."
]


agent_profiles = [
    "Agent is a new trainee reading from a script.",
    "Agent has 20+ years of experience and uses confident, professional language.",
    "Agent is friendly and conversational, tries to make the customer feel at ease.",
    "Agent speaks with a Midwest/U.S. regional tone.",
    "Agent works in the commercial claims department and is used to businesses, not individuals.",
    "Agent is clearly working remotely and occasionally pauses to look things up."
]

claim_moods = [
    "Customer is very frustrated due to a long wait.",
    "Customer is surprised by how quickly they were contacted.",
    "Customer is sad because this claim is related to a recent personal loss.",
    "Customer is confused about what is covered.",
    "Customer is trying to remain positive despite the situation.",
    "Customer is defensive and worried they may be blamed."
]

claim_complexities = [
    "The incident occurred while the customer was traveling out-of-state.",
    "There are multiple parties involved and conflicting accounts.",
    "Customer had just renewed their policy days before the loss.",
    "Photos provided are blurry and hard to verify damage.",
    "Claim involves prior similar damage that may not be covered.",
    "Customer submitted the claim late and is unsure if it will be accepted."
]



# Generate outputs
fnol_calls, adjuster_notes = [], []
start_time = time()
print("🔄 Generating FNOL and adjuster notes...\n")
for _, row in df.iterrows():
    try:
        # Get claim info
        claim_data = row.to_dict()
        row_info = "\n".join([f"{k}: {v}" for k, v in claim_data.items()])

        # Match policy by 'Policy Number'
        policy_number = claim_data.get("Policy Number")
        policy_row = policy_df[policy_df["Policy Number"] == policy_number]
        if not policy_row.empty:
            policy_info = "\n".join([f"{k}: {v}" for k, v in policy_row.iloc[0].to_dict().items()])
        else:
            policy_info = "No policy data found."

        # Prompt instructions
        tone_instruction = random.choice(tones)
        note_instruction = f"{random.choice(note_styles)} {random.choice(note_scenarios)}"
        agent_profile = random.choice(agent_profiles)
        claim_mood = random.choice(claim_moods)
        claim_context = random.choice(claim_complexities)
        instructions = f"""{tone_instruction} {agent_profile} {claim_mood} {claim_context}. 
        For notes: {note_instruction}"""

        # Build LLM prompt
        system_msg = SystemMessage(content="You are a claims assistant generating realistic transcripts and internal adjuster notes.")
        user_msg = HumanMessage(content=f"""
            You will be given insurance claim and policy details. Your job is to:

            1. Generate a FNOL call transcript between the customer and agent. {instructions}
            2. Write adjuster notes based on the call. {note_instruction}

            Claim Details:
            {row_info}

            Policy Details:
            {policy_info}

            Return a JSON object with:
            - fnol_call: the transcript
            - adjuster_notes: the internal summary
            """)

        result = structured_llm.invoke([system_msg, user_msg])
        fnol = fix_smart_quotes(result.fnol_call.strip())
        notes = result.adjuster_notes.strip()
        fnol_calls.append(fnol)
        adjuster_notes.append(notes)

    except Exception as e:
        fnol_calls.append("ERROR")
        adjuster_notes.append(f"ERROR: {e}")


# Add results to dataframe
df_result = df.copy()
df_result["fnol_call"] = fnol_calls
df_result["adjuster_notes"] = adjuster_notes



# Save the updated CSV
df_result.to_csv("C://Users//SujaySunilNagvekar//VM//GEN AI//KM//claims_with_notes.csv", index=False, encoding="utf-8")

print(f"\n✅ Done in {round(time() - start_time, 2)} seconds. File saved")


🔄 Generating FNOL and adjuster notes...


✅ Done in 7475.98 seconds. File saved


In [53]:
import pandas as pd
import sqlite3

# File path
fnol_csv = "C://Users//SujaySunilNagvekar//VM//GEN AI//KM//claims_with_notes.csv"
policy_data = "C://Users//SujaySunilNagvekar//VM//GEN AI//KM//new_policy_data.csv"
db_path = "C://Users//SujaySunilNagvekar//VM//GEN AI//KM//database.db"

In [54]:
import pandas as pd
import sqlite3

# Load the cleaned CSVs
fnol_df = pd.read_csv(fnol_csv, encoding="utf-8", engine="python", on_bad_lines="skip")
policy_df = pd.read_csv(policy_data, encoding="utf-8", engine="python", on_bad_lines="skip")

# Rename columns for fnol_data
fnol_rename_map = {
    "Claim Number": "claim_number",
    "Policy Number": "policy_number",
    "Date of Loss": "date_of_loss",
    "Time of Loss": "time_of_loss",
    "Loss Location": "loss_location",
    "Loss Location State": "loss_location_state",
    "Loss Location Zipcode": "loss_location_zipcode",
    "Date of Reporting Loss": "date_of_reporting_loss",
    "Loss cause": "loss_cause",
    "Claimant Name": "claimant_name",
    "Vehicle Make": "vehicle_make",
    "Vehicle Model": "vehicle_model",
    "Vehicle Year": "vehicle_year",
    "Damage Description": "damage_description",
    "Reported By": "reported_by",
    "Claim Status": "claim_status",
    "Claim Assigned": "claim_assigned",
    "Claim Handler Name": "claim_handler_name",
    "Litigation": "litigation",
    "Police Report": "police_report",
    "Photos/Videos": "photos_videos",
    "Repair Estimate presence": "repair_estimate_presence",
    "Repair Estimate": "repair_estimate",
    "Repair Bill": "repair_bill",
    "Towing Receipt": "towing_receipt",
    "Rental Receipt": "rental_receipt",
    "Medical & Injury Documentation": "medical_and_injury_documentation",
    "Medical Reports": "medical_reports",
    "Hospital Records": "hospital_records",
    "Medical Bill": "medical_bill",
    "Total Claim Bill": "total_claim_bill",
    "Third-Party Information": "third_party_information",
    "Subro Opportunity": "subro_opportunity",
    "Third-Party Insurance": "third_party_insurance",
    "Third-Party Claim Form": "third_party_claim_form",
    "Damage Severity": "damage_severity",
    "Injury Severity": "injury_severity",
    "Date of Payment - Repair": "date_of_payment_repair",
    "Date of Payment - Medical": "date_of_payment_medical",
    "fnol_call": "fnol_call",
    "adjuster_notes": "adjuster_notes"
}

# Rename columns for policy_data
policy_rename_map = {
    "Policy Number": "policy_number",
    "Account Number": "account_number",
    "Expiration Date": "expiration_date",
    "Effective Date": "effective_date",
    "Policy Status": "policy_status",
    "Policy Term": "policy_term",
    "Product Line": "product_line",
    "Primary Insured Name": "primary_insured_name",
    "Policy holder age": "policy_holder_age",
    "Primary Insured Address": "primary_insured_address",
    "Primary Insured location zipcode": "primary_insured_location_zipcode",
    "Primary Insured location state": "primary_insured_location_state",
    "Premium Amount": "premium_amount",
    "Underwriter Name": "underwriter_name",
    "Agent Name": "agent_name",
    "Payment Plan": "payment_plan",
    "Risk Type": "risk_type",
    "Vehicle VIN": "vehicle_vin",
    "Property Address": "property_address",
    "Cancellation Date": "cancellation_date",
    "Reason for Cancellation": "reason_for_cancellation"
}

# Apply renaming
fnol_df.rename(columns=fnol_rename_map, inplace=True)
policy_df.rename(columns=policy_rename_map, inplace=True)

# Create a new SQLite DB and insert tables
conn = sqlite3.connect("final_insurance_db.db")
fnol_df.to_sql("fnol_data", conn, if_exists="replace", index=False)
policy_df.to_sql("policy_data", conn, if_exists="replace", index=False)
conn.close()

print("✅ New database created: final_insurance_db.db")


✅ New database created: final_insurance_db.db


In [55]:
import sqlite3

conn = sqlite3.connect("final_insurance_db.db")
cursor = conn.cursor()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

print("📋 Tables in database:")
for table in tables:
    print("-", table[0])


📋 Tables in database:
- fnol_data
- policy_data


In [58]:
cursor.execute("SELECT fnol_call FROM fnol_data LIMIT 5;")
rows = cursor.fetchall()

print("\n🔍 Sample rows from fnol_data:")
for row in rows:
    print(row)



🔍 Sample rows from fnol_data:
("Agent: Thank you for contacting us today, Mr. Norman. My name is Alex, and I'll be assisting you with your claim. Can I confirm that you are reporting an incident under policy number PC171779360?\n\nCustomer: Yes, that's correct, but I'm really confused about what's covered. This whole process has been a nightmare. I've already been dealing with so much after the accident.\n\nAgent: I completely understand, and I'm here to help. First, let me confirm a few details about the incident. I see that this was reported as an animal collision that occurred on July 5th, 2023, while you were traveling in Kentucky. Is that correct?\n\nCustomer: Yes, I was out of state for work. I hit a deer on the highway, and the side of my truck is completely messed up. I also ended up with a fractured arm and had to get surgery. Does my policy cover all of these expenses?\n\nAgent: I'm sorry to hear about your injuries, Mr. Norman. Based on your policy, medical expenses related